<a href="https://colab.research.google.com/github/giftajayi/Vancouver-Crime-SQL/blob/main/Vancouver_crime_db(sql).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project focuses on importing crime data from a Google Sheets document into an SQLite database and performing various SQL operations to analyze the data. The process begins by reading the data from a Google Sheets URL into a pandas DataFrame. The DataFrame is then chunked into smaller pieces for efficient processing and inserted into a SQLite database named `vancouver_crime.db`. A table named `crime_data` is created to store various details about the crimes, such as the type, date, time, location, and geographical coordinates.


After populating the database, several SQL queries are executed to perform data analysis. This includes updating the database to handle missing values, counting crimes per neighborhood, identifying neighborhoods with the highest and lowest crime counts, and calculating average and maximum crimes per year. These operations provide insights into crime patterns and distributions within the dataset. The use of chunking ensures efficient data handling, and the structured approach of SQL queries facilitates robust data analysis and management.

In [ ]:
#@title Importing data and creating a dataframe
import pandas as pd
import sqlite3

google_sheet_url = 'https://docs.google.com/spreadsheets/d/1ZebuCQcO5t-dcXT7WP5lExa4NEwsDwexHVndh-YXnUs/edit?usp=sharing'
csv_export_url = google_sheet_url.replace('/edit?usp=sharing', '/export?format=csv')
df = pd.read_csv(csv_export_url)

In [ ]:
#@title Creating the table using SQLite
def df_chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

chunk_size = 1000
conn = sqlite3.connect('vancouver_crime.db')

create_table_query = '''
CREATE TABLE IF NOT EXISTS crime_data (
  ID INTEGER PRIMARY KEY NOT NULL,
  TYPE TEXT NOT NULL,
  YEAR INTEGER NOT NULL,
  MONTH INTEGER NOT NULL,
  DAY INTEGER NOT NULL,
  HOUR REAL,
  MINUTE REAL,
  HUNDRED_BLOCK TEXT,
  NEIGHBOURHOOD TEXT,
  X REAL,
  Y REAL,
  Latitude REAL,
  Longitude REAL
)
'''
conn.execute(create_table_query)

for chunk in df_chunker(df, chunk_size):
    chunk.to_sql('crime_data', conn, if_exists='append', index=False, method=None)

conn.close()

Loading the SQL magic and connecting to vancouver_crime database

In [ ]:
%load_ext sql

%sql sqlite:///vancouver_crime.db


Updating the neighbourhood column to change all rows that have NULL values to 'undefined'.

In [ ]:
%%sql
SELECT * from crime_data LIMIT 10;

 * sqlite:///vancouver_crime.db
Done.


ID,TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y,Latitude,Longitude
1,Other Theft,2003,5,12,16.0,15.0,9XX TERMINAL AVE,Strathcona,493906.5,5457452.47,49.26980201,-123.0837633
2,Other Theft,2003,5,7,15.0,20.0,9XX TERMINAL AVE,Strathcona,493906.5,5457452.47,49.26980201,-123.0837633
3,Other Theft,2003,4,23,16.0,40.0,9XX TERMINAL AVE,Strathcona,493906.5,5457452.47,49.26980201,-123.0837633
4,Other Theft,2003,4,20,11.0,15.0,9XX TERMINAL AVE,Strathcona,493906.5,5457452.47,49.26980201,-123.0837633
5,Other Theft,2003,4,12,17.0,45.0,9XX TERMINAL AVE,Strathcona,493906.5,5457452.47,49.26980201,-123.0837633
6,Other Theft,2003,3,26,20.0,45.0,9XX TERMINAL AVE,Strathcona,493906.5,5457452.47,49.26980201,-123.0837633
7,Break and Enter Residential/Other,2003,3,10,12.0,0.0,63XX WILTSHIRE ST,Kerrisdale,489325.58,5452817.95,49.22805078,-123.1466105
8,Mischief,2003,6,28,4.0,13.0,40XX W 19TH AVE,Dunbar-Southlands,485903.09,5455883.77,49.25555918,-123.1937252
9,Other Theft,2003,2,16,9.0,2.0,9XX TERMINAL AVE,Strathcona,493906.5,5457452.47,49.26980201,-123.0837633
10,Break and Enter Residential/Other,2003,7,9,18.0,15.0,18XX E 3RD AVE,Grandview-Woodland,495078.19,5457221.38,49.26773386,-123.067654


In [ ]:
%%sql

UPDATE crime_data
SET neighbourhood = "undefined"
WHERE neighbourhood is NULL;

 * sqlite:///vancouver_crime.db
56624 rows affected.


[]

Composing an SQL query to retrieve the total number of crime records for each neighborhood. Utilizing a Common Table Expression (CTE) to generate a derived table named crime_count_per_neighbourhood containing the necessary information. Renaming the columns in the query result as Neighbourhood and num_crimes.

In [ ]:
%%sql

WITH crime_count_per_neighbourhood AS
(
  SELECT neighbourhood,
  COUNT(neighbourhood) AS num_crimes
  FROM crime_data
  GROUP BY neighbourhood
  ORDER BY num_crimes
)
SELECT *
FROM crime_count_per_neighbourhood;


 * sqlite:///vancouver_crime.db
Done.


neighbourhood,num_crimes
Musqueam,532
Stanley Park,3775
South Cambie,5212
Shaughnessy,5426
West Point Grey,5871
Arbutus Ridge,6066
Kerrisdale,7447
Dunbar-Southlands,7746
Oakridge,8037
Killarney,10475


Developing an SQL query to determine the Neighbourhoods with the lowest and highest crime counts in a given dataset.

In [ ]:
%%sql

WITH
nighbourhoods_with_highest_crimes AS
(
  SELECT neighbourhood,
         COUNT(neighbourhood),
         MIN(neighbourhood)
  FROM crime_data
),
nighbourhoods_with_lowest_crimes AS
(
  SELECT neighbourhood,
         COUNT(neighbourhood),
         MAX(neighbourhood)
  FROM crime_data
)
SELECT * FROM nighbourhoods_with_highest_crimes
UNION
SELECT * FROM nighbourhoods_with_lowest_crimes;




 * sqlite:///vancouver_crime.db
Done.


neighbourhood,COUNT(neighbourhood),MIN(neighbourhood)
Arbutus Ridge,530652,Arbutus Ridge
undefined,530652,undefined


Determining the max average number of crimes per year with the month they happened.

In [ ]:
%%sql

WITH
max_avg_crimes_per_year AS
(
  SELECT Year, Month, COUNT(Neighbourhood) AS max_avg_crimes_per_year

  FROM crime_data
  GROUP BY Year
),
avg_crimes_per_month_per_year AS
(
  SELECT Year, Month, AVG(Neighbourhood)

  FROM crime_data

),
max_crimes_per_year AS
(
  SELECT Year, Month, MAX(Neighbourhood)
)


SELECT * FROM max_avg_crimes_per_year;

 * sqlite:///vancouver_crime.db
Done.


Year,Month,max_avg_crimes_per_year
2003,5,49928
2004,10,49279
2005,6,44664
2006,8,42322
2007,4,37692
2008,10,35414
2009,1,32172
2010,1,29688
2011,6,28573
2012,10,29222
